In [156]:
import os
api_key = os.environ.get('FLIGHT_API_KEY', '')
from typing import List

import requests
import pandas as pd

from pprint import pprint
from datetime import datetime

In [315]:
import pymongo

In [360]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["flight_app"]
mycol = mydb["timetable"]

In [361]:
cursor = mycol.find()

In [ ]:
for update in cursor:
    

{ "departure" : { "iataCode" : "DEN", "icaoCode" : "KDEN", "terminal" : null, "gate" : "B46", "baggage" : null, "delay" : "7", "scheduledTime" : "2019-09-28T22:20:00.000", "estimatedTime" : "2019-09-28T22:29:00.000", "actualTime" : "2019-09-28T22:27:00.000", "estimatedRunway" : "2019-09-28T22:27:00.000", "actualRunway" : "2019-09-28T22:27:00.000" },
 "arrival" : { "iataCode" : "PDX", "icaoCode" : "KPDX", "terminal" : null, "gate" : "E2", "baggage" : null, "delay" : null, "scheduledTime" : "2019-09-28T23:59:00.000", "estimatedTime" : "2019-09-28T23:40:00.000", "actualTime" : null, "estimatedRunway" : null, "actualRunway" : null },
 "airline" : { "name" : "United Airlines", "iataCode" : "UA", "icaoCode" : "UAL" },
 "flight" : { "number" : "980", "iataNumber" : "UA980", "icaoNumber" : "UAL980" },
 "codeshared" : null }



In [157]:
"_id" : ObjectId("5d90446a4a2467b8d5bd180a")

'9a7cdb-87fe76'

### PLAN:
##### 1.	“ SHOW ME ALL THE FLIGHT SCHEDULE THAT DEPARTS FROM DENVER FOR TODAY ON A MAP (D3.js)”
##### 2.	“AND COMPARE THE SCHEDULE WITH ITS ACTUAL DATA TO DISPLAY PROGRESS ON THE SAME MAP”

*	Display the scheduled flight time <code><strong>(/schedule route)</strong></code> and its actual data <strong><code>(/projection route)</code></strong> onto the same map
*	Estimated arrival time using Flight distance API calculation a distance (or time by speed) between flight’s current location and its destination
*	Other possibilities: time skewness (estimated flying time - actual flying time) by airlines as a performance metric

### TECHNOLOGIES:
* Local development: FLASK + MONGODB
* Web deployment: Google Cloud?

#### 1. Design for FLIGHT TRACKER API
Required fields:
* departure - iataCode
* arrival - iataCode
* flight - iataNumber
* aircraft - regNumber, iataCode
* geography - altitude, direction, latitude, longitude
* speed - horizontal, vertical, isGround
* status
* system - squawk, updated

In [195]:
base_url  = 'http://aviation-edge.com/v2/public/flights'
params = { 'key': api_key,
           'depIata': 'DEN',
#           'depIcao': '',
#           'arrIata': '',
#           'arrIcao': '',
#           'aircraftIcao': '',
#           'regNum': '',
#           'aircraftIcao24': '',
#           'airlineIata': '',
#           'airlineIcao': '',
#           'flightIata': '',
#           'flightIcao': '',
#           'flightNum': '',
#           'status': '',
#           'limit': 100
        }

resp_flight = requests.get(base_url, params=params).json()

In [196]:
len(resp_flight)

367

In [197]:
pprint(resp_flight[3])

{'aircraft': {'iataCode': 'B738',
              'icao24': '',
              'icaoCode': 'B738',
              'regNumber': 'N76503'},
 'airline': {'iataCode': 'UA', 'icaoCode': 'UAL'},
 'arrival': {'iataCode': 'LGA', 'icaoCode': 'KLGA'},
 'departure': {'iataCode': 'DEN', 'icaoCode': 'KDEN'},
 'flight': {'iataNumber': 'UA406', 'icaoNumber': 'UAL406', 'number': '406'},
 'geography': {'altitude': 11277.6,
               'direction': 88.34,
               'latitude': 42.0506,
               'longitude': -88.4931},
 'speed': {'horizontal': 1024.6, 'isGround': 0, 'vertical': 0},
 'status': 'en-route',
 'system': {'squawk': '6261', 'updated': '1569454200'}}


#### 2. Design for AIRPORT DATABASE API
Required fields:
* codeIataAirport, codeIataCity, codeIso2Country
* nameAirport, nameCountry
* latitudeAirport, longitudeAirport
* timezone(?)

In [230]:
base_url = 'https://aviation-edge.com/v2/public/airportDatabase'
params = {'key': api_key,
          'codeIso2Country': 'US'
         }

us_airports = requests.get(base_url, params=params).json()

In [17]:
len(us_airports)

2336

In [ ]:
# Display all airport results containing user-defined strings
pprint([airport for airport in us_airports if ('denver' in airport.get('timezone').lower()) and ('international' in airport.get('nameAirport').lower())])


In [34]:
# Sample data

# DENVER INTERNATIONAL AIRPORT (DIA)

#   'GMT': '-7',
#   'airportId': '1746',
#   'codeIataAirport': 'DEN',
#   'codeIataCity': 'DEN',
#   'codeIcaoAirport': 'KDEN',
#   'codeIso2Country': 'US',
#   'geonameId': '5419401',
#   'latitudeAirport': '39.84939',
#   'longitudeAirport': '-104.672844',
#   'nameAirport': 'Denver International',
#   'nameCountry': 'United States',
#   'phone': '303-342-2000',
#   'timezone': 'America/Denver'},

#### 3. Design for TIMETABLE API
Required fields ('departure'):
* departure - iataCode, scheduledTime, gate, terminal, delay (if delayed)
* arrival - iataCode, scheduledTime, gate, terminal, baggage, delay (if delayed)
* airline
* flight number (iata?)
* status

In [385]:
base_url = 'http://aviation-edge.com/v2/public/timetable'
params = {'key': api_key,
          'codeshared': 'Null',
#           'iataCode': 'DEN',
          'type': 'arrival'
         }

resp_timetable = requests.get(base_url, params=params).json()

In [386]:
len(resp_timetable)

66671

In [384]:
len([schedule for schedule in resp_timetable if schedule.get('arrival').get('iataCode') == "LAX"])

12

In [339]:
len(resp_timetable)

821

In [340]:
len([schedule for schedule in resp_timetable if schedule.get('codeshared') is None])

466

In [342]:
len(resp_timetable)

1767

In [343]:
len([schedule for schedule in resp_timetable if schedule.get('codeshared') is None])

1002

In [299]:
c = Counter()
a = [1, 2, 2, 3, 4]

In [306]:
c.update(a)

In [307]:
c

Counter({1: 3, 2: 6, 3: 3, 4: 3})

In [425]:
timetable_url = 'http://aviation-edge.com/v2/public/timetable'
tt_params = {'key': api_key,
               'iataCode': 'SJU',
             'codeshared': 'Null',
             'type': 'arrival'
             }
timetable_json = requests.get(timetable_url, params=tt_params).json()

In [428]:
airport_url = 'http://aviation-edge.com/v2/public/airportDatabase'
ap_params = {'key': api_key}
airport_json = requests.get(airport_url, params=ap_params).json()

In [430]:
for airport in airport_json:
    if 'SJU' in airport.get('codeIataAirport'):
        pprint(airport)
        break

{'GMT': '-4',
 'airportId': '7143',
 'codeIataAirport': 'SJU',
 'codeIataCity': 'SJU',
 'codeIcaoAirport': 'TJSJ',
 'codeIso2Country': 'PR',
 'geonameId': '4568127',
 'latitudeAirport': '18.437403',
 'longitudeAirport': '-66.004684',
 'nameAirport': 'Luis Munoz Marin International',
 'nameCountry': 'Puerto Rico',
 'phone': '787-791-3840',
 'timezone': 'America/Puerto_Rico'}


In [ ]:
results = []
for schedule in timetable_json:
    depart_airport = schedule.get('departure').get('iataCode')
    arrival_airport = schedule.get('arrival').get('iataCode')
    
    for airport in airport_json:
        if 'US' in airport.get('codeIso2Country'):
            if depart_airport == airport.get('codeIataAirport'):

In [392]:
airport_ref = {}
for airport in airport_json:
    airport_ref[airport.get('codeIataAirport')] = airport.get('codeIso2Country')

In [421]:
results = []
for schedule in timetable_json:
    
    depart_airport = schedule.get('departure').get('iataCode')
    arrival_airport = schedule.get('arrival').get('iataCode')
    
    if (airport_ref[depart_airport] == 'US') and (airport_ref[arrival_airport] == 'US'):
        
        if (depart_airport in major_airports) and (arrival_airport in major_airports):
    
            for airport in airport_json:
                if depart_airport == airport.get('codeIataAirport'):
                    schedule['departLat'] = float(airport.get('latitudeAirport'))
                    schedule['departLon'] = float(airport.get('longitudeAirport'))
                    schedule['departAirportTimezone'] = airport.get('timezone')
                    schedule['departCityCode'] = airport.get('codeIataCity')

                elif arrival_airport == airport.get('codeIataAirport'):
                    schedule['arrivalLat'] = float(airport.get('latitudeAirport'))
                    schedule['arrivalLon'] = float(airport.get('longitudeAirport'))
                    schedule['arrivalAirportTimezone'] = airport.get('timezone')
                    schedule['arrivalCityCode'] = airport.get('codeIataCity')

            results.append(schedule)

In [ ]:
results[:10]

In [404]:
import pandas as pd

In [412]:
major_airports.extend(list(pd.read_html('https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States')[1]['IATACode']))

In [416]:
major_airports = set(major_airports)

In [ ]:
major_airports

airportInfo.forEach(airport => {
        masterObj.forEach(item => {
            if (item.departAirportCode === airport.codeIataAirport) {
                item['departLat'] = Number(airport.latitudeAirport);
                item['departLon'] = Number(airport.longitudeAirport);
                item['departAirportTimezone'] = airport.timezone;
                item['departCityCode'] = airport.codeIataCity;
            } else if (item.arrivalAirportCode === airport.codeIataAirport) {
                item['arrivalLat'] = Number(airport.latitudeAirport);
                item['arrivalLon'] = Number(airport.longitudeAirport);
                item['arrivalAirportTimezone'] = airport.timezone;
                item['arrivalCityCode'] = airport.codeIataCity;

In [285]:
from collections import defaultdict
from collections import Counter

In [291]:
depart_DEN_dict = defaultdict(list)

In [292]:
for schedule in resp_timetable:
    if schedule.get('codeshared') is None:
        depart_DEN_dict[schedule.get('departure').get('iataCode')].append(schedule.get('arrival').get('iataCode'))

In [ ]:
depart_DEN_destinations = Counter(depart_DEN_dict['DEN'])
depart_DEN_destinations

In [427]:
timetable_json[:10]

[{'type': 'arrival',
  'status': 'scheduled',
  'departure': {'iataCode': 'STX',
   'icaoCode': 'TISX',
   'terminal': None,
   'gate': None,
   'baggage': None,
   'delay': None,
   'scheduledTime': '2019-10-01T18:00:00.000',
   'estimatedTime': None,
   'actualTime': None,
   'estimatedRunway': None,
   'actualRunway': None},
  'arrival': {'iataCode': 'SJU',
   'icaoCode': 'TJSJ',
   'terminal': 'B',
   'gate': None,
   'baggage': None,
   'delay': None,
   'scheduledTime': '2019-10-01T18:40:00.000',
   'estimatedTime': None,
   'actualTime': None,
   'estimatedRunway': None,
   'actualRunway': None},
  'airline': {'name': 'Seaborne Airlines',
   'iataCode': 'BB',
   'icaoCode': 'SBS'},
  'flight': {'number': '3557',
   'iataNumber': 'BB3557',
   'icaoNumber': 'SBS3557'},
  'codeshared': None},
 {'type': 'arrival',
  'status': 'scheduled',
  'departure': {'iataCode': 'STX',
   'icaoCode': 'TISX',
   'terminal': None,
   'gate': None,
   'baggage': None,
   'delay': None,
   'schedule

In [ ]:
pprint([schedule for schedule in resp_timetable if schedule.get('codeshared') is not None])

In [251]:
pprint([schedule for schedule in resp_flight if schedule.get('speed').get('isGround') != 0])

[{'aircraft': {'iataCode': 'B752',
               'icao24': '4CC268',
               'icaoCode': 'B752',
               'regNumber': 'TFFIC'},
  'airline': {'iataCode': 'FI', 'icaoCode': 'ICE'},
  'arrival': {'iataCode': 'KEF', 'icaoCode': 'BIKF'},
  'departure': {'iataCode': 'DEN', 'icaoCode': 'KDEN'},
  'flight': {'iataNumber': 'FI670', 'icaoNumber': 'ICE670', 'number': '670'},
  'geography': {'altitude': 7658.1,
                'direction': 44,
                'latitude': 40.672,
                'longitude': -104.117},
  'speed': {'horizontal': 755.616, 'isGround': 1, 'vertical': 8.45312},
  'status': 'en-route',
  'system': {'squawk': '1445', 'updated': '1569453605'}}]


In [209]:
match_count = 0

flight_numbers_in_timetable = {flight.get('flight').get('icaoNumber') for flight in resp_timetable}
flight_numbers_in_tracker = {flight.get('flight').get('icaoNumber') for flight in resp_flight}

for flight in resp_flight:
    if flight.get('flight').get('icaoNumber') in flight_numbers_in_timetable:
        match_count += 1

In [210]:
match_count

180

In [206]:
len(flight_numbers_in_timetable)

1019

In [208]:
len(flight_numbers_in_tracker)

367

In [132]:
def get_info_timetable(airport_code: str ='DEN', airline_code: str =None):
    
    results = []
    
    base_url = 'http://aviation-edge.com/v2/public/timetable'
    params = {'key': api_key,
              'iataCode': airport_code,
              'airline_iata': airline_code,
              'type': 'departure'
             }

    if not airline_code: del params['airline_iata']
        
    schedules = requests.get(base_url, params=params).json()
    print(f'Searched data counts: {len(schedules)}')

    for schedule in schedules:

        # departure info
        departure_iatacode = schedule['departure']['iataCode']
        departure_scheduledtime = schedule['departure']['scheduledTime']
        departure_scheduledtime = datetime.strptime(departure_scheduledtime, '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y-%m-%d %H:%M:%S')
        departure_gate = schedule['departure']['gate']
        departure_terminal = schedule['departure']['terminal']
        departure_delay = schedule['departure']['delay']

        # arrival info
        arrival_iatacode = schedule['arrival']['iataCode']
        arrival_scheduledtime = schedule['arrival']['scheduledTime']
        arrival_scheduledtime = datetime.strptime(arrival_scheduledtime, '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y-%m-%d %H:%M:%S')
        arrival_gate = schedule['arrival']['gate']
        arrival_terminal = schedule['arrival']['terminal']
        arrival_baggage = schedule['arrival']['baggage']
        arrival_delay = schedule['arrival']['delay']

        # airline info
        airline_iatacode = schedule['airline']['iataCode']
        airline_name = schedule['airline']['name']

        # flight info
        flight_iatanumber = schedule['flight']['iataNumber']
        flight_status = schedule['status']
        
        results.append(
             {
              'departure_iatacode': departure_iatacode,
              'departure_scheduledtime': departure_scheduledtime,
              'departure_gate': departure_gate,
              'departure_terminal': departure_terminal,
              'departure_delay': departure_delay,
              'arrival_iatacode': arrival_iatacode,
              'arrival_scheduledtime': arrival_scheduledtime,
              'arrival_gate': arrival_gate,
              'arrival_terminal': arrival_terminal,
              'arrival_baggage': arrival_baggage,
              'arrival_delay': arrival_delay,
              'airline_iatacode': airline_iatacode,
              'airline_name': airline_name,
              'flight_iatanumber': flight_iatanumber,
              'flight_status': flight_status  
            }
        )

    return results


In [133]:
results = get_info_timetable('DEN')

Searched data counts: 602


In [149]:
schedule_df = pd.DataFrame.from_records(results)

In [151]:
schedule_df.query("flight_iatanumber == 'WN150'")

,airline_iatacode,airline_name,arrival_baggage,arrival_delay,arrival_gate,arrival_iatacode,arrival_scheduledtime,arrival_terminal,departure_delay,departure_gate,departure_iatacode,departure_scheduledtime,departure_terminal,flight_iatanumber,flight_status
49,WN,Southwest Airlines,None,None,406,ONT,2019-09-24 22:35:00,4,35,C37,DEN,2019-09-24 21:25:00,None,WN150,active


#### Airport Database
Required fields ('departure'):

In [228]:
base_url = 'https://aviation-edge.com/v2/public/airportDatabase'
params = {'key': api_key}

airport_db = requests.get(base_url, params=params).json()

In [215]:
airport_db[4]

{'airportId': '5',
 'nameAirport': 'Apalachicola Regional',
 'codeIataAirport': 'AAF',
 'codeIcaoAirport': 'KAAF',
 'latitudeAirport': '29.733334',
 'longitudeAirport': '-84.98333',
 'geonameId': '4146153',
 'timezone': 'America/New_York',
 'GMT': '-5',
 'phone': '',
 'nameCountry': 'United States',
 'codeIso2Country': 'US',
 'codeIataCity': 'AAF'}

In [113]:
df_airport = pd.DataFrame.from_records(airport_db)
df_airport.head()

,GMT,airportId,codeIataAirport,codeIataCity,codeIcaoAirport,codeIso2Country,geonameId,latitudeAirport,longitudeAirport,nameAirport,nameCountry,phone,timezone
0,-10,1,AAA,AAA,NTGA,PF,6947726,-17.05,-145.41667,Anaa,French Polynesia,,Pacific/Tahiti
1,10,2,AAB,AAB,YARY,AU,7730796,-26.7,141.04167,Arrabury,Australia,,Australia/Brisbane
2,2,3,AAC,AAC,HEAR,EG,6297289,31.133333,33.75,El Arish International Airport,Egypt,,Africa/Cairo
3,1,4,AAE,AAE,DABB,DZ,2570559,36.821392,7.811857,Les Salines,,,Africa/Algiers
4,-5,5,AAF,AAF,KAAF,US,4146153,29.733334,-84.98333,Apalachicola Regional,United States,,America/New_York


In [115]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///airport.db', echo=True)

In [116]:
df_airport.to_sql('airports', con=engine)

2019-09-21 16:17:17,190 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-09-21 16:17:17,190 INFO sqlalchemy.engine.base.Engine ()
2019-09-21 16:17:17,191 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-09-21 16:17:17,192 INFO sqlalchemy.engine.base.Engine ()
2019-09-21 16:17:17,193 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("airports")
2019-09-21 16:17:17,193 INFO sqlalchemy.engine.base.Engine ()
2019-09-21 16:17:17,196 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE airports (
	"index" BIGINT, 
	"GMT" TEXT, 
	"airportId" TEXT, 
	"codeIataAirport" TEXT, 
	"codeIataCity" TEXT, 
	"codeIcaoAirport" TEXT, 
	"codeIso2Country" TEXT, 
	"geonameId" TEXT, 
	"latitudeAirport" TEXT, 
	"longitudeAirport" TEXT, 
	"nameAirport" TEXT, 
	"nameCountry" TEXT, 
	phone TEXT, 
	timezone TEXT
)


2019-09-21 16:17:17,197 INFO sqlalchemy.engine.base.Engine ()
2019-09-21 16:17:17,199 INFO sqlalchem

In [119]:
%%bash
open .

In [ ]:
engine.execute('SELECT * FROM airports').fetchall()

In [120]:
df = pd.read_sql_query('SELECT * FROM airports', engine)

2019-09-21 23:30:34,822 INFO sqlalchemy.engine.base.Engine SELECT * FROM airports
2019-09-21 23:30:34,822 INFO sqlalchemy.engine.base.Engine ()


In [ ]:
df

In [218]:
base_url = "http://aviation-edge.com/v2/public/routes"
params = {"key": api_key}

resp_route = requests.get(base_url, params=params).json()

In [219]:
len(resp_route)

300

In [ ]:
den_route = [route for route in resp_route if route.get('departureIata') == "OTP" ]
den_route

In [252]:
base_url = "http://aviation-edge.com/v2/public/cityDatabase"
params = {"key": api_key}

resp_cities = requests.get(base_url, params=params).json()

In [253]:
len(resp_cities)

9368

In [254]:
resp_cities[0]

{'cityId': '1',
 'nameCity': 'Anaa',
 'codeIataCity': 'AAA',
 'codeIso2Country': 'PF',
 'latitudeCity': '-17.05',
 'longitudeCity': '-145.41667',
 'timezone': 'Pacific/Tahiti',
 'GMT': '-10',
 'geonameId': '0'}

In [433]:
summary = requests.get("http://127.0.0.1:5000/get-storeddata/DEN").json()

In [434]:
summary[0]

{'airline': {'iataCode': 'B6', 'icaoCode': 'JBU', 'name': 'JetBlue Airways'},
 'arrival': {'actualRunway': None,
  'actualTime': None,
  'baggage': '2',
  'delay': None,
  'estimatedRunway': None,
  'estimatedTime': '2019-10-01T05:12:00.000',
  'gate': '8',
  'iataCode': 'JFK',
  'icaoCode': 'KJFK',
  'scheduledTime': '2019-10-01T05:43:00.000',
  'terminal': '5'},
 'arrivalAirportTimezone': 'America/New_York',
 'arrivalCityCode': 'NYC',
 'arrivalLat': 40.642334,
 'arrivalLon': -73.78817,
 'codeshared': None,
 'departAirportTimezone': 'America/Denver',
 'departCityCode': 'DEN',
 'departLat': 39.84939,
 'departLon': -104.672844,
 'departure': {'actualRunway': '2019-10-01T00:00:00.000',
  'actualTime': '2019-10-01T00:00:00.000',
  'baggage': None,
  'delay': '1',
  'estimatedRunway': '2019-10-01T00:00:00.000',
  'estimatedTime': '2019-10-01T00:09:00.000',
  'gate': 'A33',
  'iataCode': 'DEN',
  'icaoCode': 'KDEN',
  'scheduledTime': '2019-09-30T23:59:00.000',
  'terminal': None},
 'flight

In [446]:
inbounds = []
outbounds = []
inbound_airlines = []
outbound_airlines = []
inbound_delays = []
outbound_delays = []
airport = "DEN"

for route in summary:
    inbound = route.get('arrival').get('iataCode')
    outbound = route.get('departure').get('iataCode')

    if (airport == inbound):
        origin_for_inbound = outbound
        inbound_airline = route.get('airline').get('name')

        inbounds.append(origin_for_inbound)
        inbound_airlines.append(inbound_airline)
        delay = route.get('arrival').get('delay')
        
        if delay:
            delay = int(delay)
            if (delay > 60):
                delay = "delay > 60"
            elif (30 <= delay <= 60):
                delay = "30 ≥ delay ≥ 60"
            elif (delay < 30):
                delay = "delay ≤ 30"
            else:  # delay is null
                pass
            inbound_delays.append(delay)
        else:
            inbound_delays.append(delay)

    else: # airport != inbound (== outbound)
        destination_for_outbound = inbound
        outbound_airline = route.get('airline').get('name')

        outbounds.append(destination_for_outbound)
        outbound_airlines.append(outbound_airline)
        delay = route.get('arrival').get('delay')
        
        if delay:
            delay = int(delay)
            if (delay > 60):
                delay = "delay > 60"
            elif (30 <= delay <= 60):
                delay = "30 ≥ delay ≥ 60"
            elif (delay < 30):
                delay = "delay ≤ 30"
            else:  # delay is null
                pass
            outbound_delays.append(delay)
        else:
            outbound_delays.append(delay)

In [533]:
print(len(inbound_parent), len(outbound_parent), len(inbounds), len(outbounds), len(inbound_airlines), len(outbound_airlines), len(inbound_delays), len(outbound_delays))
#  = []
# inbound_airlines = []
# outbound_airlines = []
# inbound_delays = []
# outbound_delays = []

3372 4378 3372 4378 3372 4378 3372 4378


In [ ]:
# { "name": "DEN",
# "children": [
#               { "name": "inbound", "children": [ 
#                                                   { "name": "STL", "children": [
#                                                                                  {"name" : "Frontier", "children" : [
#                                                                                                                        { "None" : 1 }... ] }, ...] }, ...] }
# }

In [ ]:
parent_dict = {"name":"DEN", "children":[]}
inbound_dict = {}
airline_dict = {}
delay = {}

for inbound in inbound_parent:   # "inbound"
    inbound_dict["name"] = inbound
    inbound_dict["children"] = []
    
    if duplicate_checker(inbound_dict, "name", inbound) == False:
       
        
    airport_dict = {"children" : []}     # "SJU"
    for airport in inbounds:
        if duplicate_checker(airport_dict, category, airport)
        airport_dict['name'] = airport
        
        for airline in inbound_airlines:
            for delay in inbound_delays:
                
                

In [ ]:
def duplicate_checker(obj, category, string):
    if category == "children":
        subcategory = "name"
        for child in obj[category]:
            if child[subcategory] == string:
                return true  # duplicate found
            else:
                return false   
    else:
        subcategory = ""
        for child in obj[category]

        

In [531]:
outbound_parent = ["outbound" for _ in range(4378)]    

In [532]:
len(outbound_parent)

4378

In [479]:
master = []
for a, b, c, d in zip(inbound_parent, inbounds, inbound_airlines, inbound_delays):
    master.append({"parent" : a,
     "airport" : b,
     "airline" : c,
     "delay" : d})

In [481]:
df = pd.DataFrame.from_records(master)

In [482]:
df.head()

,airline,airport,delay,parent
0,Frontier Airlines,STL,None,inbound
1,United Airlines,AUS,None,inbound
2,Frontier Airlines,ORD,None,inbound
3,United Airlines,STL,None,inbound
4,Southwest Airlines,ABQ,None,inbound


In [500]:
df_mod = df.groupby(['parent', 'airport', 'airline', 'delay']).agg({'delay': 'count'}).sort_values(by=['airport', 'airline'])

In [508]:
df_mod = df_mod.rename(columns={"delay" : "count"})

In [515]:
df_mod = df_mod.reset_index()

In [ ]:
# { "name": "DEN",
# "children": [
#               { "name": "inbound", "children": [ 
#                                                   { "name": "STL", "children": [
#                                                                                  {"name" : "Frontier", "children" : [
#                                                                                                                        { "None" : 1 }... ] }, ...] }, ...] }
# }

In [613]:
df_mod.head()

,parent,airport,airline,delay,count
0,inbound,ABQ,Southwest Airlines,30 ≥ delay ≥ 60,1
1,inbound,ABQ,United Airlines,delay > 60,1
2,inbound,ABQ,United Airlines,delay ≤ 30,3
3,inbound,ANC,United Airlines,delay ≤ 30,2
4,inbound,ATL,Delta Air Lines,delay ≤ 30,2


In [536]:
master_dict = {}

for row in df_mod.itertuples():
    master_dict[(row[1], row[2], row[3], row[4])] = master_dict.get((row[1], row[2], row[3], row[4]),0) + row[5]
    

In [561]:
from collections import defaultdict

In [617]:
temp1_dict = defaultdict(list)
temp2_dict = defaultdict(list)
temp3_dict = {}

In [618]:

for key, value in master_dict.items():
    
    temp3_dict[key[3]] = temp3_dict.get(key[3], 0) + value
    temp2_dict[key[2]].append(temp3_dict)
    temp1_dict[key[1]].append(temp2_dict)

In [608]:
temp2_dict

defaultdict(list, {'Southwest Airlines': [{'30 ≥ delay ≥ 60': 1}]})

In [619]:
temp1_dict

defaultdict(list,
            {'ABQ': [defaultdict(list,
                          {'Southwest Airlines': [{'30 ≥ delay ≥ 60': 53,
                             'delay > 60': 23,
                             'delay ≤ 30': 227},
                            {'30 ≥ delay ≥ 60': 53,
                             'delay > 60': 23,
                             'delay ≤ 30': 227},
                            {'30 ≥ delay ≥ 60': 53,
                             'delay > 60': 23,
                             'delay ≤ 30': 227},
                            {'30 ≥ delay ≥ 60': 53,
                             'delay > 60': 23,
                             'delay ≤ 30': 227},
                            {'30 ≥ delay ≥ 60': 53,
                             'delay > 60': 23,
                             'delay ≤ 30': 227},
                            {'30 ≥ delay ≥ 60': 53,
                             'delay > 60': 23,
                             'delay ≤ 30': 227},
                            {'30 

In [537]:
master_dict

{('inbound', 'ABQ', 'Southwest Airlines', '30 ≥ delay ≥ 60'): 1,
 ('inbound', 'ABQ', 'United Airlines', 'delay > 60'): 1,
 ('inbound', 'ABQ', 'United Airlines', 'delay ≤ 30'): 3,
 ('inbound', 'ANC', 'United Airlines', 'delay ≤ 30'): 2,
 ('inbound', 'ATL', 'Delta Air Lines', 'delay ≤ 30'): 2,
 ('inbound', 'ATL', 'Frontier Airlines', 'delay ≤ 30'): 2,
 ('inbound', 'ATL', 'Southwest Airlines', 'delay > 60'): 2,
 ('inbound', 'ATL', 'Southwest Airlines', 'delay ≤ 30'): 1,
 ('inbound', 'ATL', 'United Airlines', 'delay ≤ 30'): 3,
 ('inbound', 'AUS', 'Southwest Airlines', 'delay ≤ 30'): 2,
 ('inbound', 'BNA', 'Frontier Airlines', 'delay ≤ 30'): 1,
 ('inbound', 'BNA', 'Southwest Airlines', 'delay ≤ 30'): 1,
 ('inbound', 'BNA', 'United Airlines', '30 ≥ delay ≥ 60'): 1,
 ('inbound', 'BNA', 'United Airlines', 'delay ≤ 30'): 4,
 ('inbound', 'BOS', 'JetBlue Airways', 'delay ≤ 30'): 3,
 ('inbound', 'BOS', 'Southwest Airlines', 'delay > 60'): 2,
 ('inbound', 'BOS', 'Southwest Airlines', 'delay ≤ 30'):

In [559]:
a = defaultdict(list)
b = 'dfdfdf'
a['b'].append({b : 1 })

In [560]:
a

defaultdict(list, {'b': [{'dfdfdf': 1}]})

In [ ]:
df_mod.to_dict()

In [525]:
a = "a"
b = 
{a : b}

{'a': 1}

In [451]:
inbound_family = zip(inbound_airlines, inbound_delays)
children_of_inbound = []

for airport in inbounds:
    children_of_inbound = {"name" : airport, "children" : []}
    
    delay_dict = {}
    airline_dict = {"name" : "", "children" : []}
    for airline, delay in inbound_family:
        delay_dict[delay] = delay_dict.get(delay, 0) + 1
        airline_dict['name'] = airline
        airline_dict['children'].append(delay_dict)
        
        
       

        

('STL', 'Frontier Airlines', None)


In [452]:
# { "name": "DEN",
# "children": [
#               { "name": "inbound", "children": [ 
#                                                   { "name": "STL", "children": [
#                                                                                  {"name" : "Frontier", "children" : [
#                                                                                                                        { "None" : 1 }... ] }, ...] }, ...] }
# }

In [468]:
inbound_starter = {"name" : "inbound"}
inbound_starter.setdefault("children", [])

[]

In [469]:
inbound_starter

{'name': 'inbound', 'children': []}

In [470]:
temp = {"None" : 5, ">60" : 6, "<60" : 2}

In [ ]:
outbound_parent = ["outbound" for _ in range()]